# Extract least privilege policy

In [36]:
with open('p.log','r') as f:
    linelist = f.readlines()
suffix = '(No such file or directory)\n'
res = [line for line in linelist if line.startswith('open') and not line.endswith(suffix)]

dic = {}
skiplist = ["/dev", "/proc", "/lib", "/events", "/mnt", "/run", "/lvm", "/conf", "/usr", "/bin"]
for line in res:
    path = line.split('"')[1]
#     paths = path.split('/')
    shouldSkip = False
    for start in skiplist:
        if path.startswith(start):
            shouldSkip = True
        if path == '/':
            shouldSkip = True
    if shouldSkip or path == '/etc/nshadow':
        continue
    middle = line.split('"')[2]
    permission = middle[2:].split(')')[0].split('|')
    if 'O_CLOEXEC' in permission:
        permission.remove('O_CLOEXEC')
    if 'O_NONBLOCK' in permission:
        permission.remove('O_NONBLOCK')
    if 'O_NOCTTY' in permission:
        permission.remove('O_NOCTTY')
    if 'O_DIRECTORY' in permission:
        permission.remove('O_DIRECTORY')
    dic[path] = permission[0]
# Find directories
directory = set()
for path in dic.keys():
    sep = path.split("/")[1:]
    for i in range(len(sep)-1):
        dir_path = '/'+'/'.join(sep[:i+1])
        directory.add(dir_path)

In [37]:
with open('passwd.perm', 'w') as f:
    f.write('sudo setfattr -n security.mp4 -v target /usr/bin/passwd\n')
    for di in directory:
        f.write('sudo setfattr -n security.mp4 -v dir ' + di + '\n')
    for path, permit in dic.items():
        if permit == "O_RDONLY":
            res = "sudo setfattr -n security.mp4 -v read-only "
        elif permit == "O_RDWR":
            res = "sudo setfattr -n security.mp4 -v read-write "
        elif permit == "O_WRONLY":
            res = "sudo setfattr -n security.mp4 -v write-only "
        else:
            print('ERROR! permit')
        f.write(res+path+'\n')

with open('passwd.perm.unload', 'w') as f:
    f.write('sudo setfattr -x security.mp4 /usr/bin/passwd\n')
    for di in directory:
        f.write('sudo setfattr -x security.mp4 ' + di + '\n')
    for path in dic.keys():
        f.write('sudo setfattr -x security.mp4 '+path+'\n')